In [19]:
from yahoo_fin import stock_info as si
from yahoo_fin.stock_info import get_data
from datetime import date
from datetime import timedelta
import os
from twilio.rest import Client
from dotenv import load_dotenv
import pandas as pd


load_dotenv("../twilio.env")
account_sid = os.getenv("account_sid_twilio")
auth_token = os.getenv("auth_token_twilio")
sender = os.getenv("sender_twilio")
bindings = os.getenv("bindings_twilio")
client = Client(account_sid, auth_token)



In [20]:
today = date.today()

In [21]:
yesterday = today - timedelta(days = 6)

In [22]:
yesterday

datetime.date(2021, 5, 9)

In [23]:
today = date.today()
btc = get_data("btc-usd", start_date=yesterday, end_date=today, index_as_date = True, interval="1d")
eth = get_data("eth-usd", start_date=yesterday, end_date=today, index_as_date = True, interval="1d")
spy = get_data("spy", start_date=yesterday, end_date=today, index_as_date = True, interval="1d")
gold = get_data("GLD", start_date=yesterday, end_date=today, index_as_date = True, interval="1d")
tlt = get_data("tlt", start_date=yesterday, end_date=today, index_as_date = True, interval="1d")
usd = get_data("dx-y.nyb", start_date=yesterday, end_date=today, index_as_date = True, interval="1d")

In [24]:
column = ['BTC', 'ETH', 'SPY', 'TLT', 'GLD', 'USD']

combined_df = pd.concat([btc['close'], eth['close'], spy['close'], gold['close'], tlt['close'], usd['close']], axis="columns", join="inner")
combined_df.columns = column
combined_df


,BTC,ETH,SPY,TLT,GLD,USD
2021-05-10,56704.574219,4168.701172,417.940002,172.110001,137.820007,90.209999
2021-05-11,49150.535156,3785.848633,414.209991,172.139999,137.039993,90.139999
2021-05-12,49716.191406,3715.148438,405.410004,170.429993,135.600006,90.709999
2021-05-13,49880.535156,4079.057373,410.279999,171.130005,135.809998,90.750000


In [25]:
combined_df.dropna()

,BTC,ETH,SPY,TLT,GLD,USD
2021-05-10,56704.574219,4168.701172,417.940002,172.110001,137.820007,90.209999
2021-05-11,49150.535156,3785.848633,414.209991,172.139999,137.039993,90.139999
2021-05-12,49716.191406,3715.148438,405.410004,170.429993,135.600006,90.709999
2021-05-13,49880.535156,4079.057373,410.279999,171.130005,135.809998,90.750000


In [26]:
last = combined_df.iloc[-1]
today = combined_df.iloc[-2]

In [27]:
today

BTC    49716.191406
ETH     3715.148438
SPY      405.410004
TLT      170.429993
GLD      135.600006
USD       90.709999
Name: 2021-05-12 00:00:00, dtype: float64

In [28]:
last

BTC    49880.535156
ETH     4079.057373
SPY      410.279999
TLT      171.130005
GLD      135.809998
USD       90.750000
Name: 2021-05-13 00:00:00, dtype: float64

In [29]:
daily_change = ((today - last) / last) * 100
daily_change 

BTC   -0.329475
ETH   -8.921398
SPY   -1.186993
TLT   -0.409053
GLD   -0.154621
USD   -0.044078
dtype: float64

In [30]:
daily_change.iloc[0]

-0.3294747129019281

In [31]:
twilio_message1 = ''

for index, asset in daily_change.iteritems(): 
    if asset > 1:  
        twilio_message1 = twilio_message1 + " " + f"{index} - BUY" + "\n"
    elif asset < -1:
        twilio_message1 = twilio_message1 + " " + f"{index} - SELL" + "\n"
    else:
        twilio_message1 = twilio_message1 + " " + f"{index} - HOLD" + "\n"


twilio_message1

' BTC - HOLD\n ETH - SELL\n SPY - SELL\n TLT - HOLD\n GLD - HOLD\n USD - HOLD\n'

In [32]:
print(twilio_message1)

 BTC - HOLD
 ETH - SELL
 SPY - SELL
 TLT - HOLD
 GLD - HOLD
 USD - HOLD



In [33]:
body = twilio_message1


message = client.messages.create(
                     body=body,            
                     from_= sender,
                     to =bindings
)
print(message.sid)


SM42665ac915b446739f3a5712eba2eab3
